## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place (or any two points in the Helsinki Region..).

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [5]:
# read in input files
import pandas as pd
import geopandas as gpd
from geopandas.tools import geocode

fp_actPoints = r"activity_locations.txt"
fp_shopPoints = r"shopping_centers.txt"

actPoints = pd.read_csv(fp_actPoints,sep=';')
shopPoints= pd.read_csv(fp_shopPoints,sep=';')


shopPoints.head()

,id,name,addr
0,0,Itis,"Itäkatu 1-7, 00930 Helsinki, Finlandia"
1,1,Forum,"Mannerheimintie 14–20, 00100 Helsinki, Finlandia"
2,2,Iso-omena,"Piispansilta 11, 02230 Espoo, Finlandia"
3,3,Sello,"Leppävaarankatu 3-9, 02600 Espoo, Finlandia"
4,4,Jumbo,"Vantaanportinkatu 3, 01510 Vantaa, Finlandia"


In [6]:
actPoints.head()

,id,addr
0,0,"Itälahdenkatu 5, 00210 Helsinki, Finlandia"
1,1,"Raitamäenkuja 3, 00950 Helsinki, Finlandia"


In [16]:
# Geocode activity locations
geoShop = geocode(shopPoints['addr'], provider='nominatim',user_agent='autogis_21',timeout=10)
geoAct  = geocode(actPoints['addr'], provider='nominatim',user_agent='autogis_21',timeout=10)
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
#raise NotImplementedError()

In [17]:
print(geoShop)

                    geometry  \
0  POINT (25.08294 60.21170)   
1  POINT (24.93828 60.16878)   
2  POINT (24.73779 60.16294)   
3  POINT (24.81042 60.21752)   
4  POINT (24.96282 60.29245)   
5  POINT (24.97904 60.18702)   
6  POINT (24.93068 60.19875)   

                                             address  
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...  
1  Salaattiasema, 14-20, Mannerheimintie, Keskust...  
2  Sports Academy, 11, Piispansilta, Matinkylän k...  
3  Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...  
4  Stockmann, 3, Vantaanportinkatu, Vantaanportti...  
5  Silta, 5, Hermannin rantatie, Verkkosaari, Kal...  
6  Apteekki 51, 2b, Firdonkatu, Keski-Pasila, Pas...  


In [18]:
print(geoAct)

                    geometry  \
0  POINT (24.88255 60.14837)   
1  POINT (25.10530 60.22109)   

                                             address  
0  5, Itälahdenkatu, Vattuniemi, Lauttasaari, Ete...  
1  3, Raitamäenkuja, Vartioharju, Vartiokylä, Itä...  


In [19]:
type(geoShop)

geopandas.geodataframe.GeoDataFrame

In [20]:
# Check coordinate reference system information (re-project/define if needed)
geoShop.crs == geoAct.crs
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
#raise NotImplementedError()

True

In [21]:
geoAct.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [24]:
geoShop = geoShop.to_crs(epsg=3879)
geoAct = geoAct.to_crs(epsg=3879)
geoAct.crs

<Projected CRS: EPSG:3879>
Name: ETRS89 / GK25FIN
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Finland - 24.5°E to 25.5°E onshore nominal
- bounds: (24.5, 59.94, 25.5, 68.9)
Coordinate Operation:
- name: Finland Gauss-Kruger zone 25
- method: Transverse Mercator
Datum: European Terrestrial Reference System 1989
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [29]:
# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:
from shapely.ops import nearest_points

def get_nearest_shopCenter(row, shopp_gdf):
    """Find the cearest shopping center to two random locations in Helsinki region"""
    
    Dest_Muligeometry = shopp_gdf['geometry'].unary_union
    nearest_geoms = nearest_points(row['geometry'],Dest_Muligeometry)
    nearest_data = shopp_gdf.loc[shopp_gdf["geometry"] == nearest_geoms[1]]
    nearest_shopp = nearest_data['geometry'].values[0]
    return nearest_shopp

geoAct["nearest_shopp"] = geoAct.apply(get_nearest_shopCenter,shopp_gdf=geoShop,axis=1)
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
#raise NotImplementedError()

In [30]:
geoAct.head()

,geometry,address,nearest_shopp
0,POINT (25493475.664 6670608.813),"5, Itälahdenkatu, Vattuniemi, Lauttasaari, Ete...",POINT (25496573.54241596 6672878.360095562)
1,POINT (25505836.315 6678710.054),"3, Raitamäenkuja, Vartioharju, Vartiokylä, Itä...",POINT (25504598.60162313 6677662.108634275)


In [36]:
geoShop

,geometry,address
0,POINT (25504598.602 6677662.109),"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va..."
1,POINT (25496573.542 6672878.360),"Salaattiasema, 14-20, Mannerheimintie, Keskust..."
2,POINT (25485440.532 6672255.563),"Sports Academy, 11, Piispansilta, Matinkylän k..."
3,POINT (25489491.076 6678322.265),"Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä..."
4,POINT (25497943.932 6686656.982),"Stockmann, 3, Vantaanportinkatu, Vantaanportti..."
5,POINT (25498837.156 6674909.983),"Silta, 5, Hermannin rantatie, Verkkosaari, Kal..."
6,POINT (25496155.285 6676217.986),"Apteekki 51, 2b, Firdonkatu, Keski-Pasila, Pas..."


In [38]:
#Activate geometry column with nearest shopping center 
geoAct = geoAct.set_geometry('nearest_shopp')

In [40]:

#test: shops = geoShop[geoShop.intersects(geoAct['nearest_shopp'])]
#test: geoShop.intersects(geoAct['nearest_shopp'].unary_union)
geoAct = gpd.sjoin(geoAct,geoShop,op='intersects',how='inner')
#test: pip_mask = data.within(southern.at[0, 'geometry']) 
#test: pip_data = data.loc[pip_mask]

In [41]:
geoAct.head()

,geometry,address_left,nearest_shopp,index_right,address_right
0,POINT (25493475.664 6670608.813),"5, Itälahdenkatu, Vattuniemi, Lauttasaari, Ete...",POINT (25496573.542 6672878.360),1,"Salaattiasema, 14-20, Mannerheimintie, Keskust..."
1,POINT (25505836.315 6678710.054),"3, Raitamäenkuja, Vartioharju, Vartiokylä, Itä...",POINT (25504598.602 6677662.109),0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va..."


In [55]:
print(pd.__version__)

1.1.5


In [63]:
geoAct = geoAct.rename(columns={'index_right':'id'})
geoAct1= pd.merge(geoAct,shopPoints, on = 'id')

Can you think of other application cases for the nearest neighbour analysis?

In [67]:
geoAct1.head()

,geometry,address_left,nearest_shopp,id,address_right,name,addr
0,POINT (25493475.664 6670608.813),"5, Itälahdenkatu, Vattuniemi, Lauttasaari, Ete...",POINT (25496573.542 6672878.360),1,"Salaattiasema, 14-20, Mannerheimintie, Keskust...",Forum,"Mannerheimintie 14–20, 00100 Helsinki, Finlandia"
1,POINT (25505836.315 6678710.054),"3, Raitamäenkuja, Vartioharju, Vartiokylä, Itä...",POINT (25504598.602 6677662.109),0,"Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...",Itis,"Itäkatu 1-7, 00930 Helsinki, Finlandia"


In [ ]:
print(geoAct1[['address_left','name']])

YOUR ANSWER HERE

In [72]:
print("Shopping center closest to home is "+geoAct1.at[0,'name'])

Shopping center closest to home is Forum


In [73]:
print("Shopping center closest to work is "+geoAct1.at[1,'name'])

Shopping center closest to work is Itis
